<a href="https://colab.research.google.com/github/CristValen/Acciones-RNR/blob/main/nps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelBinarizer
from imblearn.over_sampling import SMOTE
import numpy as np
import pandas as pd

# Establecer la semilla del generador de números aleatorios
np.random.seed(42)

# Supongamos que 'df' es tu DataFrame y que 'clase' es tu variable objetivo
df = pd.read_csv('tu_archivo.csv')

# Crear las variables objetivo y características
X = df.drop('clase', axis=1)
y = df['clase']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Aplicar SMOTE al conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Crear el modelo de Random Forest
modelo = RandomForestClassifier(random_state=42)

# Definir la grilla de hiperparámetros para la búsqueda aleatoria
param_dist = {
    'n_estimators': [50, 100, 200, 300, 400],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Realizar la búsqueda aleatoria de hiperparámetros
busqueda_aleatoria = RandomizedSearchCV(estimator=modelo,
                                        param_distributions=param_dist,
                                        n_iter=100,
                                        cv=3,
                                        verbose=2,
                                        random_state=42,
                                        n_jobs=-1)

# Entrenar el modelo con los mejores hiperparámetros encontrados
busqueda_aleatoria.fit(X_train_res, y_train_res)

# Obtener las probabilidades de las predicciones en el conjunto de prueba
probas = busqueda_aleatoria.predict_proba(X_test)

# Definir tus umbrales personalizados aquí. Por ejemplo:
umbrales = [0.4, 0.3, 0.3]  # Un umbral para cada clase

# Aplicar los umbrales a las probabilidades para obtener las predicciones finales
predicciones = np.array([busqueda_aleatoria.classes_[i] for i in np.argmax(probas >= umbrales, axis=1)])

# Imprimir un informe de clasificación
print("Informe de clasificación:")
print(classification_report(y_test, predicciones))

# Calcular e imprimir la matriz de confusión con los nombres de las clases
cm = confusion_matrix(y_test, predicciones)
cm_df = pd.DataFrame(cm,
                     index=[i for i in busqueda_aleatoria.classes_],
                     columns=[i for i in busqueda_aleatoria.classes_])
print("Matriz de confusión:")
print(cm_df)

# Imprimir la precisión del modelo
print("Precisión del modelo:")
print(accuracy_score(y_test, predicciones))

# Calcular e imprimir la puntuación AUC-ROC para cada clase
lb = LabelBinarizer()
lb.fit(y_test)
y_test_lb = lb.transform(y_test)
predicciones_lb = lb.transform(predicciones)
print("Puntuación AUC-ROC para cada clase:")
for i in range(len(lb.classes_)):
    print(f"{lb.classes_[i]}: {roc_auc_score(y_test_lb[:, i], predicciones_lb[:, i])}")






In [ ]:
#########nuevos

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, roc_curve, auc
from sklearn.preprocessing import LabelBinarizer
from imblearn.over_sampling import SMOTE
import numpy as np
import pandas as pd

# Establecer la semilla del generador de números aleatorios
np.random.seed(42)

# Supongamos que 'df' es tu DataFrame y que 'clase' es tu variable objetivo
df = pd.read_csv('tu_archivo.csv')

# Crear las variables objetivo y características
X = df.drop('clase', axis=1)
y = df['clase']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Aplicar SMOTE al conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Crear el modelo de Random Forest
modelo = RandomForestClassifier(random_state=42)

# Definir la grilla de hiperparámetros para la búsqueda aleatoria
param_dist = {
    'n_estimators': [50, 100, 200, 300, 400],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Realizar la búsqueda aleatoria de hiperparámetros
busqueda_aleatoria = RandomizedSearchCV(estimator=modelo,
                                        param_distributions=param_dist,
                                        n_iter=100,
                                        cv=3,
                                        verbose=2,
                                        random_state=42,
                                        n_jobs=-1)

# Entrenar el modelo con los mejores hiperparámetros encontrados
busqueda_aleatoria.fit(X_train_res, y_train_res)

# Obtener las probabilidades de las predicciones en el conjunto de prueba
probas = busqueda_aleatoria.predict_proba(X_test)

# Imprimir un informe de clasificación
print("Informe de clasificación:")
print(classification_report(y_test, predicciones))

# Calcular e imprimir la matriz de confusión con los nombres de las clases
cm = confusion_matrix(y_test, predicciones)
cm_df = pd.DataFrame(cm,
                     index=[i for i in busqueda_aleatoria.classes_],
                     columns=[i for i in busqueda_aleatoria.classes_])
print("Matriz de confusión:")
print(cm_df)

# Imprimir la precisión del modelo
print("Precisión del modelo:")
print(accuracy_score(y_test, predicciones))

# Calcular e imprimir la puntuación AUC-ROC para cada clase
lb = LabelBinarizer()
lb.fit(y_test)
y_test_lb = lb.transform(y_test)
predicciones_lb = lb.transform(predicciones)
print("Puntuación AUC-ROC para cada clase:")
for i in range(len(lb.classes_)):
    print(f"{lb.classes_[i]}: {roc_auc_score(y_test_lb[:, i], predicciones_lb[:, i])}")

# Función para calcular el valor KS
def calc_ks(data):
    data['bucket']=(data['probability'].rank(pct=True)*10).astype(int)
    grouped=data.groupby('bucket',as_index=True)
    kstable=grouped.min().probability.to_frame(name='min_prob')
    kstable['max_prob']=grouped.max().probability
    kstable['bads']=grouped.sum().bad
    kstable['goods']=grouped.sum().good
    kstable=kstable.reset_index()
    kstable['bad_rate']=kstable.bads/(kstable.bads+kstable.goods)
    kstable['ks']=(kstable.bads/kstable.bads.sum()).cumsum()-(kstable.goods/kstable.goods.sum()).cumsum()
    ks_value=kstable.ks.abs().max()
    return ks_value

# Función para calcular el coeficiente Gini
def calc_gini(y_verdadero, y_prob):
    fpr, tpr, _ = roc_curve(y_verdadero, y_prob)
    roc_auc = auc(fpr, tpr)
    gini = 2*roc_auc - 1
    return gini

# Calcular e imprimir el valor KS y el coeficiente Gini para cada clase
print("Valor KS y coeficiente Gini para cada clase:")
for i in range(len(lb.classes_)):
    data = pd.DataFrame({'label': y_test_lb[:, i], 'probability': probas[:, i]})
    data['good']=(data['label']==0).astype(int)
    data['bad']=(data['label']==1).astype(int)
    ks = calc_ks(data)
    gini = calc_gini(y_test_lb[:, i], probas[:, i])
    print(f"{lb.classes_[i]}: KS = {ks}, Gini = {gini}")



In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelBinarizer
from imblearn.over_sampling import SMOTE
import numpy as np
import pandas as pd

# Establecer la semilla del generador de números aleatorios
np.random.seed(42)

# Supongamos que 'df' es tu DataFrame y que 'clase' es tu variable objetivo
df = pd.read_csv('tu_archivo.csv')

# Crear las variables objetivo y características
X = df.drop('clase', axis=1)
y = df['clase']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Aplicar SMOTE al conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Crear el modelo SVM
modelo = SVC(probability=True, random_state=42)

# Definir la grilla de hiperparámetros para la búsqueda aleatoria
param_dist = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf', 'poly', 'sigmoid']
}

# Realizar la búsqueda aleatoria de hiperparámetros
busqueda_aleatoria = RandomizedSearchCV(estimator=modelo,
                                        param_distributions=param_dist,
                                        n_iter=100,
                                        cv=3,
                                        verbose=2,
                                        random_state=42,
                                        n_jobs=-1)

# Entrenar el modelo con los mejores hiperparámetros encontrados
busqueda_aleatoria.fit(X_train_res, y_train_res)

# Obtener las probabilidades de las predicciones en el conjunto de prueba
probas = busqueda_aleatoria.predict_proba(X_test)

# Definir tus umbrales personalizados aquí. Por ejemplo:
umbrales = [0.4, 0.3, 0.3]  # Un umbral para cada clase

# Aplicar los umbrales a las probabilidades para obtener las predicciones finales
predicciones = np.array([busqueda_aleatoria.classes_[i] for i in np.argmax(probas >= umbrales, axis=1)])

# Imprimir un informe de clasificación
print("Informe de clasificación:")
print(classification_report(y_test, predicciones))

# Calcular e imprimir la matriz de confusión con los nombres de las clases
cm = confusion_matrix(y_test, predicciones)
cm_df = pd.DataFrame(cm,
                     index=[i for i in busqueda_aleatoria.classes_],
                     columns=[i for i in busqueda_aleatoria.classes_])
print("Matriz de confusión:")
print(cm_df)

# Imprimir la precisión del modelo
print("Precisión del modelo:")
print(accuracy_score(y_test, predicciones))

# Calcular e imprimir la puntuación AUC-ROC para cada clase
lb = LabelBinarizer()
lb.fit(y_test)
y_test_lb = lb.transform(y_test)
predicciones_lb = lb.transform(predicciones)
print("Puntuación AUC-ROC para cada clase:")
for i in range(len(lb.classes_)):
    print(f"{lb.classes_[i]}: {roc_auc_score(y_test_lb[:, i], predicciones_lb[:, i])}")


In [ ]:
from pyspark.sql.functions import to_date

# Asumiendo que tu DataFrame se llama df y tus columnas son 'ope' y 'conta'
df = df.withColumn("ope", to_date(df['ope'], "yyyy-MM-dd"))
df = df.withColumn("conta", to_date(df['conta'], "yyyy-MM-dd"))

# Agrupar por la columna 'ope'
df_grouped_by_ope = df.groupBy("ope").count()

# Agrupar por la columna 'conta'
df_grouped_by_conta = df.groupBy("conta").count()

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelBinarizer
import numpy as np
import pandas as pd

# Establecer la semilla del generador de números aleatorios
np.random.seed(42)

# Supongamos que 'df' es tu DataFrame y que 'clase' es tu variable objetivo
df = pd.read_csv('tu_archivo.csv')

# Crear las variables objetivo y características
X = df.drop('clase', axis=1)
y = df['clase']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear una instancia del clasificador Gradient Boosting
modelo = GradientBoostingClassifier(random_state=42)

# Definir la grilla de hiperparámetros para la búsqueda en grilla
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
}

# Realizar la búsqueda en grilla de hiperparámetros
busqueda_en_grilla = GridSearchCV(estimator=modelo,
                                  param_grid=param_grid,
                                  cv=3,
                                  verbose=2,
                                  n_jobs=-1)

# Entrenar el modelo con los mejores hiperparámetros encontrados
busqueda_en_grilla.fit(X_train, y_train)

# Predecir las clases para el conjunto de prueba
y_pred = busqueda_en_grilla.predict(X_test)

# Definir tus umbrales personalizados aquí. Por ejemplo:
umbrales = [0.4, 0.3, 0.3]  # Un umbral para cada clase

# Aplicar los umbrales a las probabilidades para obtener las predicciones finales
probas = busqueda_en_grilla.predict_proba(X_test)
predicciones = np.array([busqueda_en_grilla.classes_[i] for i in np.argmax(probas >= umbrales, axis=1)])

# Imprimir un informe de clasificación
print("Informe de clasificación:")
print(classification_report(y_test, predicciones))

# Calcular e imprimir la matriz de confusión con los nombres de las clases
cm = confusion_matrix(y_test, predicciones)
cm_df = pd.DataFrame(cm,
                     index=[i for i in busqueda_en_grilla.classes_],
                     columns=[i for i in busqueda_en_grilla.classes_])
print("Matriz de confusión:")
print(cm_df)

# Imprimir la precisión del modelo
print("Precisión del modelo:")
print(accuracy_score(y_test, predicciones))

# Calcular e imprimir la puntuación AUC-ROC para cada clase
lb = LabelBinarizer()
lb.fit(y_test)
y_test_lb = lb.transform(y_test)
predicciones_lb = lb.transform(predicciones)
print("Puntuación AUC-ROC para cada clase:")
for i in range(len(lb.classes_)):
    print(f"{lb.classes_[i]}: {roc_auc_score(y_test_lb[:, i], predicciones_lb[:, i])}")


In [ ]:


import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, label_binarize

# Establecer la semilla para la reproducibilidad
np.random.seed(0)

# Supongamos que X es tu conjunto de datos e y son las etiquetas correspondientes

# Crear el codificador
le = LabelEncoder()

# Ajustar el codificador y transformar las etiquetas
y = le.fit_transform(y)

# Convertir los datos a float32
X = X.astype(np.float32)
y = y.astype(np.float32)

# División de los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Creación del modelo MLP más profundo
model = Sequential()
model.add(Dense(50, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(3, activation='softmax'))  # Tres neuronas en la capa de salida para los tres niveles de la variable dependiente

# Compilación del modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Asignación de pesos a las clases (supongamos que class_weights es un diccionario que contiene los pesos)
class_weights = {0: 1., 1: 1., 2: 1.}  # Modifica esto según tus necesidades

# Entrenamiento del modelo
model.fit(X_train, y_train, epochs=100, verbose=0, class_weight=class_weights)

# Predicción en el conjunto de prueba
y_pred_proba = model.predict(X_test)

# Definir los umbrales de decisión para cada clase
thresholds = [0.5, 0.5, 0.5]  # Modifica esto según tus necesidades

y_pred = np.zeros_like(y_pred_proba)
for i in range(3):
    y_pred[:, i] = (y_pred_proba[:, i] > thresholds[i]).astype(int)

y_pred = np.argmax(y_pred, axis=1)  # Convertir las predicciones one-hot a etiquetas

# Calcular la matriz de confusión para todas las categorías
confusion = confusion_matrix(y_test, y_pred)

# Crear un DataFrame para visualizar la matriz de confusión con los nombres de las clases
class_names = ['detractor', 'neutro', 'promotor']  # Modifica esto según tus necesidades
confusion_df = pd.DataFrame(confusion, index=class_names, columns=class_names)

print("Confusion Matrix: \n", confusion_df)

# Calcular las métricas para cada categoría
for i in range(3):
    precision = precision_score(y_test == i, y_pred == i)
    recall = recall_score(y_test == i, y_pred == i)
    f1 = f1_score(y_test == i, y_pred == i)

    # Calcular la curva ROC y el AUC-ROC
    fpr, tpr, _ = roc_curve(y_test == i, y_pred_proba[:, i])
    roc_auc = auc(fpr, tpr)

    print(f"Clase {class_names[i]}:")
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1 Score: ", f1)
    print("ROC AUC Score: ", roc_auc)


In [ ]:
# Calcular las métricas ponderadas para cada categoría
for i in range(3):
    precision = precision_score(y_test == i, y_pred == i, average='weighted')
    recall = recall_score(y_test == i, y_pred == i, average='weighted')
    f1 = f1_score(y_test == i, y_pred == i, average='weighted')

    print(f"Clase {class_names[i]} ponderada:")
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1 Score: ", f1)

### a mano
from sklearn.metrics import f1_score

# Supongamos que 'y_test' son tus etiquetas verdaderas e 'y_pred' son tus predicciones
f1_scores = [f1_score(y_test == i, y_pred == i) for i in range(3)]
class_counts = [sum(y_test == i) for i in range(3)]
total_count = len(y_test)

weighted_f1_score = sum(f1 * count for f1, count in zip(f1_scores, class_counts)) / total_count

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, roc_curve, auc, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import numpy as np
import pandas as pd

# Establecer la semilla del generador de números aleatorios
np.random.seed(42)

# Supongamos que 'df' es tu DataFrame y que 'clase' es tu variable objetivo
df = pd.read_csv('tu_archivo.csv')

# Crear las variables objetivo y características
X = df.drop('clase', axis=1)
y = df['clase']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear una instancia del clasificador Gradient Boosting
modelo = GradientBoostingClassifier(random_state=42)

# Definir la grilla de hiperparámetros para la búsqueda en grilla
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
}

# Realizar la búsqueda en grilla de hiperparámetros
busqueda_en_grilla = GridSearchCV(estimator=modelo,
                                  param_grid=param_grid,
                                  cv=3,
                                  verbose=2,
                                  n_jobs=-1)

# Entrenar el modelo con los mejores hiperparámetros encontrados
busqueda_en_grilla.fit(X_train, y_train)

# Obtener las probabilidades de las predicciones en el conjunto de prueba
probas = busqueda_en_grilla.predict_proba(X_test)

# Definir tus umbrales personalizados aquí. Por ejemplo:
umbrales = [0.4, 0.3, 0.3]  # Un umbral para cada clase

# Aplicar los umbrales a las probabilidades para obtener las predicciones finales
predicciones = np.array([busqueda_en_grilla.classes_[i] for i in np.argmax(probas >= umbrales, axis=1)])

# Imprimir un informe de clasificación
print("Informe de clasificación:")
print(classification_report(y_test, predicciones))

# Calcular e imprimir la matriz de confusión con los nombres de las clases
cm = confusion_matrix(y_test, predicciones)
cm_df = pd.DataFrame(cm,
                     index=[i for i in busqueda_en_grilla.classes_],
                     columns=[i for i in busqueda_en_grilla.classes_])
print("Matriz de confusión:")
print(cm_df)

# Imprimir la precisión del modelo
print("Precisión del modelo:")
print(accuracy_score(y_test, predicciones))

# Calcular e imprimir la puntuación AUC-ROC para cada clase
lb = LabelBinarizer()
lb.fit(y_test)
y_test_lb = lb.transform(y_test)
predicciones_lb = lb.transform(predicciones)
print("Puntuación AUC-ROC para cada clase:")
for i in range(len(lb.classes_)):
    print(f"{lb.classes_[i]}: {roc_auc_score(y_test_lb[:, i], predicciones_lb[:, i])}")

# Función para calcular el valor KS
def calc_ks(data):
    data['good']=(data['label']==0).astype(int)
    data['bad']=(data['label']==1).astype(int)
    data['bucket']=(data['probability'].rank(pct=True)*10).astype(int)
    grouped=data.groupby('bucket',as_index=True)
    kstable=grouped.min().probability.to_frame(name='min_prob')
    kstable['max_prob']=grouped.max().probability
    kstable['bads']=grouped.sum().bad
    kstable['goods']=grouped.sum().good
    kstable=kstable.reset_index()
    kstable['bad_rate']=kstable.bads/(kstable.bads+kstable.goods)
    kstable['ks']=(kstable.bads/kstable.bads.sum()).cumsum()-(kstable.goods/kstable.goods.sum()).cumsum()
    ks_value=kstable.ks.abs().max()
    return ks_value

# Función para calcular el coeficiente Gini
def calc_gini(y_verdadero, y_prob):
    fpr, tpr, _ = roc_curve(y_verdadero, y_prob)
    roc_auc = auc(fpr, tpr)
    gini = 2*roc_auc - 1
    return gini

# Calcular e imprimir el valor KS y el coeficiente Gini para cada clase
print("Valor KS y coeficiente Gini para cada clase:")
for i in range(len(lb.classes_)):
    data = pd.DataFrame({'label': y_test_lb[:, i], 'probability': probas[:, i]})
    data['good']=(data['label']==0).astype(int)
    data['bad']=(data['label']==1).astype(int)
    ks = calc_ks(data)
    gini = calc_gini(y_test_lb[:, i], probas[:, i])
    print(f"{lb.classes_[i]}: KS = {ks}, Gini = {gini}")



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, roc_curve, auc
from sklearn.preprocessing import LabelEncoder, label_binarize

# Establecer la semilla para la reproducibilidad
np.random.seed(0)

# Supongamos que X2 es tu conjunto de datos e y2 son las etiquetas correspondientes
X2 = X.copy()
y2 = y.copy()

# Crear el codificador
le2 = LabelEncoder()

# Ajustar el codificador y transformar las etiquetas
y2 = le2.fit_transform(y2)

# Convertir los datos a float32
X2 = X2.astype(np.float32)
y2 = y2.astype(np.float32)

# División de los datos en conjuntos de entrenamiento y prueba
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=0)

# Creación del modelo MLP más profundo
model2 = Sequential()
model2.add(Dense(50, activation='relu', input_shape=(X_train.shape[1],)))
model2.add(Dense(50, activation='relu'))
model2.add(Dense(50, activation='relu'))
model2.add(Dense(3, activation='softmax'))  # Tres neuronas en la capa de salida para los tres niveles de la variable dependiente

# Compilación del modelo
model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Asignación de pesos a las clases (supongamos que class_weights es un diccionario que contiene los pesos)
class_weights = {0: 1., 1: 1., 2: 1.}  # Modifica esto según tus necesidades

# Entrenamiento del modelo
model2.fit(X_train2, y_train2, epochs=100, verbose=0, class_weight=class_weights)

# Predicción en el conjunto de prueba
y_pred_proba2 = model.predict(X_test)

# Definir los umbrales de decisión para cada clase
thresholds = [0.5, 0.5, 0.5]  # Modifica esto según tus necesidades

y_pred2 = np.zeros_like(y_pred_proba)
for i in range(3):
    y_pred[:, i] = (y_pred_proba[:, i] > thresholds[i]).astype(int)

y_pred = np.argmax(y_pred, axis=1)  # Convertir las predicciones one-hot a etiquetas

# Calcular la matriz de confusión para todas las categorías
confusion = confusion_matrix(y_test, y_pred)

# Crear un DataFrame para visualizar la matriz de confusión con los nombres de las clases
class_names = ['detractor', 'neutro', 'promotor']  # Modifica esto según tus necesidades
confusion_df = pd.DataFrame(confusion, index=class_names, columns=class_names)

print("Confusion Matrix: \n", confusion_df)

# Calcular las métricas para cada categoría
for i in range(3):
    precision = precision_score(y_test == i, y_pred == i)
    recall = recall_score(y_test == i, y_pred == i)
    f1 = f1_score(y_test == i, y_pred == i)

    # Calcular la curva ROC y el AUC-ROC
    fpr, tpr, _ = roc_curve(y_test == i, y_pred_proba[:, i])
    roc_auc = auc(fpr, tpr)

    print(f"Clase {class_names[i]}:")
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1 Score: ", f1)
    print("ROC AUC Score: ", roc_auc)

# Función para calcular el valor KS
def calc_ks(data):
    data['good']=(data['label']==0).astype(int)
    data['bad']=(data['label']==1).astype(int)
    data['bucket']=(data['probability'].rank(pct=True)*10).astype(int)
    grouped=data.groupby('bucket',as_index=True)
    kstable=grouped.min().probability.to_frame(name='min_prob')
    kstable['max_prob']=grouped.max().probability
    kstable['bads']=grouped.sum().bad
    kstable['goods']=grouped.sum().good
    kstable=kstable.reset_index()
    kstable['bad_rate']=kstable.bads/(kstable.bads+kstable.goods)
    kstable['ks']=(kstable.bads/kstable.bads.sum()).cumsum()-(kstable.goods/kstable.goods.sum()).cumsum()
    ks_value=kstable.ks.abs().max()
    return ks_value

# Función para calcular el coeficiente Gini
def calc_gini(y_verdadero, y_prob):
    fpr, tpr, _ = roc_curve(y_verdadero, y_prob)
    roc_auc = auc(fpr, tpr)
    gini = 2*roc_auc - 1
    return gini

# Calcular e imprimir el valor KS y el coeficiente Gini para cada clase
print("Valor KS y coeficiente Gini para cada clase:")
for i in range(len(class_names)):
    data = pd.DataFrame({'label': (y_test == i).astype(int), 'probability': y_pred_proba[:, i]})
    data['good']=(data['label']==0).astype(int)
    data['bad']=(data['label']==1).astype(int)
    ks = calc_ks(data)
    gini = calc_gini((y_test == i).astype(int), y_pred_proba[:, i])
    print(f"{class_names[i]}: KS = {ks}, Gini = {gini}")


    from scipy.optimize import differential_evolution

# Definir la función objetivo para la optimización
def objective(thresholds):
    # Aplicar los umbrales a las probabilidades para obtener las predicciones finales
    y_pred = np.zeros_like(y_pred_proba2)
    for i in range(3):
        y_pred[:, i] = (y_pred_proba2[:, i] > thresholds[i]).astype(int)
    y_pred = np.argmax(y_pred, axis=1)

    # Calcular las métricas
    roc_auc_scores = []
    ks_values = []
    gini_values = []
    for i in range(len(class_names)):
        data = pd.DataFrame({'label': (y_test2 == i).astype(int), 'probability': y_pred_proba2[:, i]})
        data['good']=(data['label']==0).astype(int)
        data['bad']=(data['label']==1).astype(int)
        ks = calc_ks(data)
        gini = calc_gini((y_test2 == i).astype(int), y_pred_proba2[:, i])
        fpr, tpr, _ = roc_curve((y_test2 == i).astype(int), y_pred_proba2[:, i])
        roc_auc = auc(fpr, tpr)

        roc_auc_scores.append(roc_auc)
        ks_values.append(ks)
        gini_values.append(gini)

    # Calcular el promedio de las métricas
    avg_roc_auc = np.mean(roc_auc_scores)
    avg_ks = np.mean(ks_values)
    avg_gini = np.mean(gini_values)

    # La función objetivo es el negativo del promedio de las métricas porque queremos maximizarlas
    return -(avg_roc_auc + avg_ks + avg_gini) / 3

# Definir los límites para los umbrales
bounds = [(0, 1)] * 3

# Realizar la optimización
result = differential_evolution(objective, bounds)

# Imprimir los mejores umbrales encontrados
print("Mejores umbrales: ", result.x)

